In [1]:
import pandas as pd
import os
import json

purchase_data = os.path.join("Resources","purchase_data2.json")
    
with open(purchase_data) as datafile:
    data = json.load(datafile)
purchase_data_pd = pd.DataFrame(data)    

purchase_data_pd.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35
1,21,Male,12,Dawne,3.36,Aidaira26
2,17,Male,5,Putrid Fan,2.63,Irim47
3,17,Male,123,Twilight's Carver,2.55,Irith83
4,22,Male,154,Feral Katana,4.11,Philodil43


In [2]:
#purchase_data_pd['SN'].nunique()

In [2]:
Players = purchase_data_pd ["SN"].value_counts()

TotalPlayers = len (Players)
print("Total Players:  ",TotalPlayers)

Total Players:   74


In [3]:
Items = purchase_data_pd ["Item Name"].value_counts()

NumberOfItems = len (Items)
NumberOfPurchases = (len (purchase_data_pd))
TotalRevenue = purchase_data_pd["Price"].sum()
AveragePrice = TotalRevenue/NumberOfPurchases

PurchaseAnalysis = pd.DataFrame({"Number of Unique Items":[NumberOfItems],"Average Price":[AveragePrice],"Number of Purchases":[NumberOfPurchases],"Total Revenue":[TotalRevenue]})

PurchaseAnalysis["Average Price"] = PurchaseAnalysis["Average Price"].map("${0:,.2f}".format)
PurchaseAnalysis["Total Revenue"] = PurchaseAnalysis["Total Revenue"].map("${0:,.2f}".format)

#This command is needed to rearrange the columns in the correct order -- they print out of order otherwise  
PurchaseAnalysis = PurchaseAnalysis[["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"]]    
PurchaseAnalysis




,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,63,$2.92,78,$228.10


In [4]:
Players_df= purchase_data_pd[["SN","Gender","Age"]]
#print (Players_df.shape)
Players_df.drop_duplicates(inplace=True)
#print (Players_df.shape)

total_gender = Players_df["Gender"].count()
male = Players_df["Gender"].value_counts()["Male"]
percent_male = male/total_gender * 100
female = Players_df["Gender"].value_counts()["Female"]
percent_female = female/total_gender * 100
non_gender_specific = total_gender - male - female
percent_ng_specific = non_gender_specific/total_gender * 100

GenderDemo = pd.DataFrame({"Gender":["Male", "Female", "Other/Non-Disclosed"],"Percentage of Players":[percent_male, percent_female, percent_ng_specific],"Total Count":[male, female, non_gender_specific]})


GenderDemo["Percentage of Players"] = GenderDemo["Percentage of Players"].map("{0:,.2f}".format)

GenderDemo


C:\Users\sboxberg\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Gender,Percentage of Players,Total Count
0,Male,81.08,60
1,Female,17.57,13
2,Other/Non-Disclosed,1.35,1


In [5]:
male_purchases = purchase_data_pd[purchase_data_pd["Gender"] == "Male"]
female_purchases = purchase_data_pd[purchase_data_pd["Gender"] == "Female"]
other_purchases = purchase_data_pd[purchase_data_pd["Gender"] == "Other / Non-Disclosed"]

male_purchase_count = len (male_purchases)
female_purchase_count = len (female_purchases)
other_purchase_count = len (other_purchases)

male_purchase_ave = male_purchases["Price"].mean()
female_purchase_ave = female_purchases["Price"].mean()
other_purchase_ave = other_purchases["Price"].mean()

male_purchase_total = male_purchases["Price"].sum()
female_purchase_total = female_purchases["Price"].sum()
other_purchase_total = other_purchases["Price"].sum()

male_purchase_norm = male_purchase_total/male
female_purchase_norm = female_purchase_total/female
other_purchase_norm = other_purchase_total/non_gender_specific

GenderPurchase = pd.DataFrame({"Gender":["Male", "Female", "Other/Non-Disclosed"],
                               "Purchase Count":[male_purchase_count, female_purchase_count, other_purchase_count],
                               "Average Purchase Price":[male_purchase_ave, female_purchase_ave, other_purchase_ave],
                               "Total Purchase Value":[male_purchase_total, female_purchase_total, other_purchase_total],
                               "Normalized Totals":[male_purchase_norm, female_purchase_norm, other_purchase_norm]})

# Format
GenderPurchase["Average Purchase Price"] = GenderPurchase["Average Purchase Price"].map("${0:,.2f}".format)
GenderPurchase["Total Purchase Value"] = GenderPurchase["Total Purchase Value"].map("${0:,.2f}".format)
GenderPurchase["Normalized Totals"] = GenderPurchase["Normalized Totals"].map("${0:,.2f}".format)

#Needed to rearrange the columns
GenderPurchase = GenderPurchase[["Gender", "Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
GenderPurchase

,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
0,Male,64,$2.88,$184.60,$3.08
1,Female,13,$3.18,$41.38,$3.18
2,Other/Non-Disclosed,1,$2.12,$2.12,$2.12


In [6]:
bins = [0, 10, 14, 19, 24, 29, 34, 39, 999]
bin_names = ['<10', '10 - 14', '15 - 19', '20 - 24', '25 - 29', '30 - 34', '35 - 39', '40+']

#Added the extra cutting of Player data due to the stupid normalization total :)

purchase_data_pd["Age Group"] = pd.cut(purchase_data_pd["Age"], bins, labels=bin_names)
Players_df["Age Group"] = pd.cut(Players_df["Age"], bins, labels=bin_names)

purchase_groups_by_age = purchase_data_pd.groupby("Age Group")
players_groups_by_age = Players_df.groupby ("Age Group")

PurchaseCount = purchase_groups_by_age["Age Group"].count()
PlayerCount = players_groups_by_age["Age Group"].count()

TotalPurchase = purchase_groups_by_age["Price"].sum()
AveragePurchasePrice = TotalPurchase/PurchaseCount

NormalizedTotal = TotalPurchase/PlayerCount

PurchaseAnalysisAge = pd.DataFrame({"Purchase Count":PurchaseCount,
                                    "Average Purchase Price": AveragePurchasePrice, "Total Purchase Value":TotalPurchase,
                                   "Normalized Totals":NormalizedTotal})

PurchaseAnalysisAge["Average Purchase Price"] = PurchaseAnalysisAge["Average Purchase Price"].map("${0:,.2f}".format)
PurchaseAnalysisAge["Total Purchase Value"] = PurchaseAnalysisAge["Total Purchase Value"].map("${0:,.2f}".format)
PurchaseAnalysisAge["Normalized Totals"] = PurchaseAnalysisAge["Normalized Totals"].map("${0:,.2f}".format)

#Needed for correct column arrangement on output
PurchaseAnalysisAge = PurchaseAnalysisAge [["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
PurchaseAnalysisAge


C:\Users\sboxberg\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Group,,,,
<10,5,$2.76,$13.82,$2.76
10 - 14,3,$2.99,$8.96,$2.99
15 - 19,11,$2.76,$30.41,$2.76
20 - 24,36,$3.02,$108.89,$3.20
25 - 29,9,$2.90,$26.11,$3.26
30 - 34,7,$1.98,$13.89,$2.31
35 - 39,6,$3.56,$21.37,$3.56
40+,1,$4.65,$4.65,$4.65


In [7]:
purchase_groups_by_name = purchase_data_pd.groupby("SN")

In [8]:
TotalPurchase_by_name = purchase_groups_by_name["Price"].sum()
PurchaseCount_by_name = purchase_groups_by_name["SN"].count()
AveragePrice_by_name = TotalPurchase_by_name/PurchaseCount_by_name

TopSpenders = pd.DataFrame({"Purchase Count":PurchaseCount_by_name,
                            "Average Purchase Price": AveragePrice_by_name,
                            "Total Purchase Value":TotalPurchase_by_name})

TopSpenders.sort_values("Total Purchase Value", ascending=False, inplace = True)

#order
TopSpenders=TopSpenders[["Purchase Count","Average Purchase Price", "Total Purchase Value"]]

#format
TopSpenders["Average Purchase Price"] = TopSpenders["Average Purchase Price"].map("${0:,.2f}".format)
TopSpenders["Total Purchase Value"] = TopSpenders["Total Purchase Value"].map("${0:,.2f}".format)


TopSpenders.head(5)


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Sundaky74,2,$3.71,$7.41
Aidaira26,2,$2.56,$5.13
Eusty71,1,$4.81,$4.81
Chanirra64,1,$4.78,$4.78
Alarap40,1,$4.71,$4.71


In [9]:
purchase_groups_by_ID = purchase_data_pd.groupby("Item ID")

TotalPurchase_by_ID = purchase_groups_by_ID["Price"].sum()
PurchaseCount_by_ID = purchase_groups_by_ID["Item ID"].count()
ItemName_by_ID = purchase_groups_by_ID["Item Name"].first()
Price_by_ID = purchase_groups_by_ID["Price"].sum()/purchase_groups_by_ID["Price"].count()

#I struggled to get the Item Name and Price to pull in.  I finally decided to do that silly operation on price and the .first
#on ItemName to get it so I could add it to my dataframe correctly.  I'm sure I'm missing something obvious, but after hours
#of struggles, I was pretty pumped that I was able to figure out a workaround. :)

TopPurchases = pd.DataFrame({"Item Name": ItemName_by_ID,
                             "Purchase Count": PurchaseCount_by_ID, "Item Price":Price_by_ID,
                            "Total Purchase Value":TotalPurchase_by_ID})

#Need to sort the list for the final section first before formating -- just capturing an extra copy now before I format
TopPurchases_by_Value = TopPurchases.sort_values(["Total Purchase Value"], ascending = False)

TopPurchases.sort_values(["Purchase Count", "Total Purchase Value"], ascending=[False,False], inplace = True)


#order
TopPurchases=TopPurchases[["Item Name","Purchase Count", "Item Price","Total Purchase Value"]]

#format
TopPurchases["Item Price"] = TopPurchases["Item Price"].map("${0:,.2f}".format)
TopPurchases["Total Purchase Value"] = TopPurchases["Total Purchase Value"].map("${0:,.2f}".format)

TopPurchases.head(5)


,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
94,Mourning Blade,3,$3.64,$10.92
117,"Heartstriker, Legacy of the Light",2,$4.71,$9.42
93,Apocalyptic Battlescythe,2,$4.49,$8.98
90,Betrayer,2,$4.12,$8.24
154,Feral Katana,2,$4.11,$8.22


In [10]:
#order
TopPurchases_by_Value=TopPurchases_by_Value[["Item Name","Purchase Count", "Item Price","Total Purchase Value"]]

#format
TopPurchases_by_Value["Item Price"] = TopPurchases_by_Value["Item Price"].map("${0:,.2f}".format)
TopPurchases_by_Value["Total Purchase Value"] = TopPurchases_by_Value["Total Purchase Value"].map("${0:,.2f}".format)

TopPurchases_by_Value.head(5)

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
94,Mourning Blade,3,$3.64,$10.92
117,"Heartstriker, Legacy of the Light",2,$4.71,$9.42
93,Apocalyptic Battlescythe,2,$4.49,$8.98
90,Betrayer,2,$4.12,$8.24
154,Feral Katana,2,$4.11,$8.22
